# 0. aug test

In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

def augment_one_image(img_file_path, dst_dir, num):
    image_data_gen =  ImageDataGenerator(
        rotation_range=150,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range = 0.1,
        shear_range=0,
        horizontal_flip=True
    )

    p = img_file_path
    img = load_img( p )

    img_array = img_to_array(img)
    img_array = img_array.reshape( (1, img_array.shape[0], img_array.shape[1], img_array.shape[2]) )

    i = 0
    prefix='aug__'+os.path.basename(p).replace('.png','')
    for batch in image_data_gen.flow(img_array, batch_size=1,save_to_dir=dst_dir, save_prefix=prefix, save_format='png'):
        i = i+1
        if i>=num:
            break


2024-07-19 21:12:03.900489: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-19 21:12:03.902766: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 21:12:03.950838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 21:12:03.951952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 21:12:04.795869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# 1. no-padding-resize

In [38]:
import os

dist = {}
src = 'V4/'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '')
    if y not in dist:
        dist[y] = 0
    dist[y] += 1

In [39]:
dist

{'red_blood_cell': 2910,
 'gametocyte': 8,
 'schizont': 11,
 'ring': 39,
 'trophozoite': 5}

In [40]:
from shutil import copyfile
from tqdm import *


dst = 'V7/'

for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    fp_dst = os.path.join(dst, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell':
        copyfile(fp_src,fp_dst)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2973/2973 [00:00<00:00, 11451.30it/s]


In [41]:
# one process, too slow
from shutil import copyfile
from tqdm import *


dst = 'V7/'

for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    fp_dst = os.path.join(dst, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell' or y=='difficult':
        continue
    else:
        num_now = dist[y]
        num = dist['red_blood_cell']//num_now
        augment_one_image(fp_src, dst, num)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2973/2973 [03:16<00:00, 15.14it/s]


In [42]:
# check aug result
import os

dist = {}
src = 'V7/'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '').split('_')[0]
    if y not in dist:
        dist[y] = 0
    dist[y] += 1
    
dist

{'red': 2910,
 'trophozoite': 2816,
 'gametocyte': 2844,
 'schizont': 2854,
 'ring': 2876}

In [6]:
import time
from multiprocessing import Process
import os


thnum = 8

def runs(sub_task_para_list):
    for xxx in sub_task_para_list:
        fp_src, dst, num = xxx
        augment_one_image(fp_src, dst, num)
        
n = len(task_para_list)//thnum
for i in range(thnum):
    p = Process(target=runs, args=(task_para_list[i*n:(i+1)*n],))  # target进程执行的任务, args传参数（元祖）
    p.start() 

NameError: name 'task_para_list' is not defined